In [1]:
from src.preprocess import load_training_set, get_test_video_set
from src.evaluation import pick_test_segments, generate_test_segments, evaluate_segments, evaluate
from src.Video import Video
from src.Segment import Segment
from src.demo import test_inspect_widgets, chunks
from src import search
import ipywidgets as widgets
from IPython.display import HTML
import cv2 

# Demo

### Configuration

In [2]:
NUM_VIDEOS      = 200
GRID_SIZE       = 2
BINS            = [18, 25]
HIST_FRAME_SKIP = 5
REFRESH         = False

### Index videos

In [3]:
video_index = load_training_set(range(1, NUM_VIDEOS+1), GRID_SIZE, BINS, HIST_FRAME_SKIP, force_refresh=REFRESH)

### Create test collections

In [ ]:
all_histograms, all_labels = get_test_video_set(NUM_VIDEOS, GRID_SIZE, BINS, n=100)
test_index = list(chunks(all_histograms, 20))
test_labels = list(chunks(all_labels, 20))

<br><br>

# TOC

 - [Inspect test set](#Inspect-test-set)
 - [Manual search](#Manual-search)
 - [Evaluation](#Evaluation)

<br>

# Inspect test set

In [17]:
test_set_id_widget, test_set_vid_widget = test_inspect_widgets(test_labels)
display(test_set_id_widget, test_set_vid_widget)

Dropdown(description='Test set:', options=(('-', '-'), ('Test set 1', 0), ('Test set 2', 1), ('Test set 3', 2)…

Dropdown(description='Video:', disabled=True, options=(' ', '-', ' '), value=' ')

In [19]:
video = [x for x in video_index if x.name == test_set_vid_widget.value[0]][0]
start_time = int(round(test_set_vid_widget.value[1] / video.frame_rate))
end_time = int(round(test_set_vid_widget.value[2] / video.frame_rate))

HTML("""<video id="vid" width="480" height="270" controls><source src="data/movies/%s" type="video/mp4"></video>
<script>
v = document.getElementById('vid');
v.currentTime = %d;v.addEventListener('timeupdate', function(){ if(this.currentTime >= %d) { this.pause(); }});
v.play();
</script>""" % (video.name, start_time, end_time))

<br><br>

# Manual Search

In [20]:
test_segment_index = test_labels[test_set_id_widget.value].index(test_set_vid_widget.value)
hists = test_index[test_set_id_widget.value][test_segment_index]
found = search.knownImageSearch(hists, video_index, cv2.HISTCMP_CHISQR_ALT, 5, HIST_FRAME_SKIP, [0])
found

('00032.mp4', 4930.5)

In [21]:
video = [x for x in video_index if x.name == found[0]][0]
start_time = int(round(found[1] / video.frame_rate))
end_time = start_time + int(round(20 * video.frame_rate))

HTML("""<video id="vid_found" width="480" height="270" controls><source src="data/movies/%s" type="video/mp4"></video>
<script>
v = document.getElementById('vid_found');
v.currentTime = %d;v.addEventListener('timeupdate', function(){ if(this.currentTime >= %d) { this.pause(); }});
v.play();
</script>""" % (video.name, start_time, end_time))

<br><br>

# Evaluation

In [ ]:
%%time
test_set = test_index[test_set_id_widget.value]
results = []

for i, test_segment in enumerate(test_set):
    print("\rSearching segment {}/{}".format(i+1, len(test_set), len(test_segment)), end='', flush=True)
    results.append(search.knownImageSearch(test_segment, video_index, cv2.HISTCMP_CHISQR_ALT, 5, HIST_FRAME_SKIP, [0]))

print()

Searching segment 2/20

In [ ]:
movie_results, start_frame_dist = evaluate(results, test_labels[test_set_id_widget.value])

fractions = (movie_results[0] / movie_results[2]*100 if movie_results[2] > 0 else 0, \
             movie_results[1] / movie_results[0]*100 if movie_results[0] > 0 else 0)

print("TEST RESULTS (test set #{:d})".format(test_set_id_widget.value+1))

print("\nCorrect video: {:d} / {:d} ({:.1f}%)".format(movie_results[0], movie_results[2], fractions[0]))
print("Inside fragment: {:d} / {:d} ({:.1f}%)".format(movie_results[1], movie_results[0], fractions[1]))
print("Average distance to center of segment: {:.0f} +/- {:.0f} frames (approx. {:.1f} sec)".format(
    start_frame_dist[0], start_frame_dist[1], start_frame_dist[0]/30))

<br><br>

In [ ]:
list(zip(test_labels[test_set_id_widget.value], results))